In [1]:
from bs4 import BeautifulSoup
import requests
import csv
from csv import writer

import pickle
import sys
import pandas as pd

In [2]:
url= "http://reports.ieso.ca/public/Adequacy2/PUB_Adequacy2_20220603.xml"

In [3]:
page = requests.get(url,allow_redirects=True)

In [4]:
soup = BeautifulSoup(page.text, 'lxml')
#lists = soup.find_all('div', class_="c1head")

/home/bhavya/miniconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [5]:
title= soup.find('doctitle').string
title

'Adequacy Report'

In [6]:
out_struc=pd.read_csv('adequacy_202208161044.csv')
print(out_struc.columns)
df= pd.DataFrame(columns=out_struc.columns)

Index(['mkt_date', 'mkt_he', 'dacp_flag', 'file_ver', 'created_at', 'ont_cap',
       'ont_ene', 'int_nuc_cap', 'int_nuc_out', 'int_nuc_off', 'int_nuc_sch',
       'int_gas_cap', 'int_gas_out', 'int_gas_off', 'int_gas_sch',
       'int_hyd_cap', 'int_hyd_out', 'int_hyd_for', 'int_hyd_off',
       'int_hyd_sch', 'int_win_cap', 'int_win_out', 'int_win_for',
       'int_win_sch', 'int_sol_cap', 'int_sol_out', 'int_sol_for',
       'int_sol_sch', 'int_bio_cap', 'int_bio_out', 'int_bio_off',
       'int_bio_sch', 'int_oth_cap', 'int_oth_out', 'int_oth_off',
       'int_oth_sch', 'int_tot_out', 'int_tot_off', 'int_tot_sch',
       'imp_man_off', 'imp_man_sch', 'imp_min_off', 'imp_min_sch',
       'imp_mic_off', 'imp_mic_sch', 'imp_new_off', 'imp_new_sch',
       'imp_que_off', 'imp_que_sch', 'imp_tot_off', 'imp_tot_sch',
       'imp_tot_est', 'imp_tot_cap', 'ont_bot_cap', 'ont_reg', 'ont_tot_sup',
       'ont_for_dem', 'ont_peak_dem', 'ont_avg_dem', 'ont_emb_wind',
       'ont_emb_sol', 'ont

In [7]:
# Helper Function to fetch the child columns
def fill_cap_list(element):
    xml_cap=element.find("capacities").find_all("capacity")
    cap=[]
    for element in xml_cap:
        if element.find("energymw")==None:
            cap.append(None)
            continue
        cap.append(element.find("energymw").text)
    return cap
def fill_out_list(element):
    xml_out=element.find("outages").find_all("outage")
    out=[]
    for element in xml_out:
        if element.find("energymw")==None:
            out.append(None)
            continue
        out.append(element.find("energymw").text)
    return out
def fill_off_list(element):
    xml_off=element.find("offers").find_all("offer")
    off=[]
    for element in xml_off:
        if element.find("energymw")==None:
            off.append(None)
            continue
        off.append(element.find("energymw").text)
    return off
def fill_sch_list(element):
    xml_sch=element.find("schedules").find_all("schedule")
    sch=[]
    for element in xml_sch:
        if element.find("energymw")==None:
            sch.append(None)
            continue
        sch.append(element.find("energymw").text)
    return sch
def fill_for_list(element):
    xml_arr=element.find("forecasts").find_all("forecast")
    arr=[]
    for element in xml_arr:
        if element.find("energymw")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymw").text)
    return arr
def fill_fore_list(element):
    xml_arr=element.find("forecastenergies").find_all("forecastenergy")
    arr=[]
    for element in xml_arr:
        if element.find("energymwhr")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymwhr").text)
    return arr
def fill_off_for_list(element):
    xml_arr=element.find("offerforecasts").find_all("offerforecast")
    arr=[]
    for element in xml_arr:
        if element.find("energymw")==None:
            arr.append(None)
            continue
        arr.append(element.find("energymw").text)
    return arr

In [8]:
# Ontario Capacity
xml_cap=soup.find("capacities").find_all("capacity")
ont_cap=[]
for element in xml_cap:
    if element.find("energymw")==None:
        ont_cap.append(None)
        continue
    ont_cap.append(element.find("energymw").text)
df['ont_cap']=ont_cap

In [9]:
# Ontario Energy
xml_ene=soup.find("energies").find_all("energy")
ont_ene=[]
for element in xml_ene:
    ont_ene.append(element.find("energymwhr").text)
df['ont_ene']=ont_ene

In [10]:
# Internal Resources
xml_int=soup.find("internalresources").find_all("internalresource")
for int_element in xml_int:
    if int_element.find("fueltype").text=='Nuclear':
        df['int_nuc_cap']=fill_cap_list(int_element)
        df['int_nuc_out']=fill_out_list(int_element)
        df['int_nuc_off']=fill_off_list(int_element)
        df['int_nuc_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Gas':
        df['int_gas_cap']=fill_cap_list(int_element)
        df['int_gas_out']=fill_out_list(int_element)
        df['int_gas_off']=fill_off_list(int_element)
        df['int_gas_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Hydro':
        df['int_hyd_cap']=fill_cap_list(int_element)
        df['int_hyd_out']=fill_out_list(int_element)
        df['int_hyd_for']=fill_fore_list(int_element)
        df['int_hyd_off']=fill_off_list(int_element)
        df['int_hyd_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Wind':
        df['int_win_cap']=fill_cap_list(int_element)
        df['int_win_out']=fill_out_list(int_element)
        df['int_win_for']=fill_for_list(int_element)
        df['int_win_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Solar':
        df['int_sol_cap']=fill_cap_list(int_element)
        df['int_sol_out']=fill_out_list(int_element)
        df['int_sol_for']=fill_for_list(int_element)
        df['int_sol_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Biofuel':
        df['int_bio_cap']=fill_cap_list(int_element)
        df['int_bio_out']=fill_out_list(int_element)
        df['int_bio_off']=fill_off_list(int_element)
        df['int_bio_sch']=fill_sch_list(int_element)
    elif int_element.find("fueltype").text=='Other':
        df['int_oth_cap']=fill_cap_list(int_element)
        df['int_oth_out']=fill_out_list(int_element)
        df['int_oth_off']=fill_off_for_list(int_element)
        df['int_oth_sch']=fill_sch_list(int_element)
    else:
        print('Unknown Fueltype')
# Total Internal Resources
xml_tot_int=soup.find("internalresources").find("totalinternalresources")
df['int_tot_out']=fill_out_list(xml_tot_int)
df['int_tot_off']=fill_off_for_list(xml_tot_int)
df['int_tot_sch']=fill_sch_list(xml_tot_int)

In [11]:
pd.set_option('display.max_columns', None)
# pd.reset_option(‘max_columns’)
df.head()

,mkt_date,mkt_he,dacp_flag,file_ver,created_at,ont_cap,ont_ene,int_nuc_cap,int_nuc_out,int_nuc_off,int_nuc_sch,int_gas_cap,int_gas_out,int_gas_off,int_gas_sch,int_hyd_cap,int_hyd_out,int_hyd_for,int_hyd_off,int_hyd_sch,int_win_cap,int_win_out,int_win_for,int_win_sch,int_sol_cap,int_sol_out,int_sol_for,int_sol_sch,int_bio_cap,int_bio_out,int_bio_off,int_bio_sch,int_oth_cap,int_oth_out,int_oth_off,int_oth_sch,int_tot_out,int_tot_off,int_tot_sch,imp_man_off,imp_man_sch,imp_min_off,imp_min_sch,imp_mic_off,imp_mic_sch,imp_new_off,imp_new_sch,imp_que_off,imp_que_sch,imp_tot_off,imp_tot_sch,imp_tot_est,imp_tot_cap,ont_bot_cap,ont_reg,ont_tot_sup,ont_for_dem,ont_peak_dem,ont_avg_dem,ont_emb_wind,ont_emb_sol,ont_disp_cap,ont_disp_bid,ont_disp_on,ont_disp_off,ont_hdr_bid,ont_hdr_sch,ont_hdr_cur,exp_man_bid,exp_man_sch,exp_min_bid,exp_min_sch,exp_mic_bid,exp_mic_sch,exp_new_bid,exp_new_sch,exp_que_bid,exp_que_sch,exp_tot_bid,exp_tot_sch,exp_tot_cap,grh_tot,grh_min_10n,grh_min_10s,grh_lfu,grh_add,total_req,cap_excess,energy_excess,offered_cap_excess,res_not_sch,imp_not_sch
0,NaN,NaN,NaN,NaN,NaN,37250,23591,13081,3490,9807,9616,10679,1752,3902,361,7737,761,5506,6349,4189,4943,4268,675,623,478,478,0,0,315,303,12,12,17,0,0,0,11554,20744,14800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,37250,23518,13081,3490,9807,9616,10679,1752,3904,361,7737,761,5401,6133,4011,4943,4236,707,675,478,478,0,0,315,303,12,12,17,0,0,0,11523,20562,14674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,37250,23485,13081,3490,9807,9616,10679,1752,3905,361,7737,761,5402,6127,3928,4943,4269,673,485,478,478,0,0,315,303,12,10,17,0,0,0,11555,20523,14400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,37250,23379,13081,3490,9807,9616,10679,1752,3917,361,7737,761,5362,6126,3927,4943,4335,607,566,478,478,0,0,315,303,12,10,17,0,0,0,11620,20469,14480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,37250,23437,13081,3490,9807,9616,10679,1752,3920,361,7737,761,5365,6128,4116,4943,4282,661,606,478,477,1,1,315,303,12,12,17,0,0,0,11567,20528,14711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data_report={'deliveryhour':[]}
for i in range(len(xml_cap)):
    delivery_hour= xml_cap[i].find("deliveryhour").text
    energy_mw= xml_cap[i].find("energymw").text
    
    data_report['deliveryhour'].append([delivery_hour,energy_mw])
print(data_report)
    

{'deliveryhour': [['1', '37250'], ['2', '37250'], ['3', '37250'], ['4', '37250'], ['5', '37250'], ['6', '37250'], ['7', '37250'], ['8', '37250'], ['9', '37250'], ['10', '37250'], ['11', '37250'], ['12', '37250'], ['13', '37250'], ['14', '37250'], ['15', '37250'], ['16', '37250'], ['17', '37250'], ['18', '37250'], ['19', '37250'], ['20', '37250'], ['21', '37250'], ['22', '37250'], ['23', '37250'], ['24', '37250']]}


In [13]:
#print(page.text)